# Build (People)
Builds the `people` table from a flat file database using names as the primary key which are assumed to be unique strings for this project. Other notebooks should be used for adding additional records to the table.

### Import libraries

In [1]:
# Basic stack
from datetime import datetime

# Web Stack
import json
import requests

# Database stack
import sqlite3

# Data Science stack
import pandas as pd

### User defined variables

In [2]:
db = "../../db.sqlite3"
people = "../../data/databases/people.xlsx"

### User defined functions

In [3]:
def update_people(c, name, column_name, column_value):
    """ Updates EE table values of a given column with a given value using the 
            ENTITY ID as the selection criteria for the update.

        C - A cursor
        ENTITY ID - An entity id from EarthExplorer
        COLUMN NAME - A column name within the EE table to be updated
        COLUMN VALUE - A value to update the column
    """
    sql_string = f"UPDATE people SET {column_name} = \"{column_value}\" WHERE name = \"{name}\""
    c.execute(sql_string)


def database_activity(db, name, column_name, column_value, activity='update'):
    """ Wrapper function for activities related to database management.
            Currently only supports UPDATING based on EarthExplorer 
            Entity ID values.

        Is dependent on the UPDATE EE function above.

        DB - A database target
        ENTITY ID - An EarthExplorer Entity ID uniquely identifying
            the record of interest.
        COLUMN NAME - The column to be updated
        COLUMN VALUE - The value of the column to be updated
        ACTIVITY - The activity to be carried out. Currently only
            supports UPDATE.
    """
    conn = sqlite3.connect(db)
    conn.enable_load_extension(True)
    conn.execute("SELECT load_extension('mod_spatialite')")
    c = conn.cursor()

    if activity == 'update':
        try:
            update_people(c, name, column_name, column_value)
        except Exception as e:
            print("Exception: {} was raised for {}".format(e, row['name']))
    else:
        print("Your activity method is not currently supported.")
    
    conn.commit()
    conn.close()

def insert_person(c, name):
    """ Inserts People into the People Data Table. People's 
            names are the Primary Key for the datatable since
            they are assumed to be unique for this effort.

        C - A cursor
        PERSON - A Person's Name
    """
    sql_string = f"INSERT INTO people(name) VALUES (\"{name}\")"
    c.execute(sql_string)

### Load data

In [4]:
df = pd.read_excel(people)
df.head()

,name,email,organization,sub_organization,location
0,Christin Khan,christin.khan@noaa.gov,NOAA,Northeast Fisheries Science Center,"Woods Hole, Massachusetts"
1,Tim White,NaN,NaN,NaN,NaN
2,Kim Goetz,kim.goetz@noaa.gov,NOAA,Alaska Fisheries Science Center,"Seattle, Washington"
3,Tyler Aldrich,tyler.aldrich@noaa.gov,NOAA Officer Corps,Northeast Fisheries Science Center,"Woods Hole, Massachusetts"
4,Meredith Sackett,meredith.sackett@noaa.gov,NOAA,Northeast Fisheries Science Center,"Woods Hole, Massachusetts"


### Drop table
This is for demonstration purposes

In [5]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()
c.execute('''DROP TABLE IF EXISTS people''')
conn.commit()
conn.close()

### Create `people` table

In [6]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

c.execute('''
    CREATE TABLE IF NOT EXISTS people(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name VARCHAR(20),
        email VARCHAR(30),
        organization VARCHAR(25),
        sub_organization VARCHAR(50),
        location VARCHAR(50)
    )
''')

conn.commit()
conn.close()

### Insert EarthExplorer Entity IDs into `ee` table

In [7]:
conn = sqlite3.connect(db)
conn.enable_load_extension(True)
conn.execute("SELECT load_extension('mod_spatialite')")

c = conn.cursor()

for i, row in df.iterrows():
    try:
        insert_person(c, row['name'])
    except Exception as e:
        print("Exception: {} was raised for {}".format(e, row['name']))
print("Done inserting people into table!")
conn.commit()
conn.close()

Done inserting people into table!


### Update `people` records from spreadsheet

In [8]:
columns = df.columns[1:]
for i, row in df.iterrows():
    pid = row['name']
    print("Updating information for: {}".format(pid))
    for column in columns:
        database_activity(db, pid, column, row[column])

Updating information for: Christin Khan
Updating information for: Tim White
Updating information for: Kim Goetz
Updating information for: Tyler Aldrich
Updating information for: Meredith Sackett
Updating information for: Kathleen Leonard
Updating information for: Davida Streett


# End